{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 🌌 Kerr Black Hole Visualization\n",
    "\n",
    "This notebook demonstrates how to use BlackHole-Sim to visualize a rotating (Kerr) black hole, similar to **Gargantua** from the movie *Interstellar*.\n",
    "\n",
    "## Contents\n",
    "1. [Black Hole Properties](#1.-Black-Hole-Properties)\n",
    "2. [Rendering an Image](#2.-Rendering-an-Image)\n",
    "3. [Exploring Parameter Space](#3.-Exploring-Parameter-Space)\n",
    "4. [Physics Analysis](#4.-Physics-Analysis)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Import libraries\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "from matplotlib.colors import LogNorm\n",
    "from ipywidgets import interact, FloatSlider, IntSlider\n",
    "\n",
    "# Import our black hole simulation package\n",
    "import bhsim\n",
    "\n",
    "print(f\"BlackHole-Sim version: {bhsim.__version__}\")\n",
    "\n",
    "# Set up plotting style\n",
    "plt.style.use('dark_background')\n",
    "plt.rcParams['figure.figsize'] = (10, 10)\n",
    "plt.rcParams['font.size'] = 12"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Black Hole Properties\n",
    "\n",
    "Let's create a near-extremal Kerr black hole (spin a/M = 0.99) and examine its properties."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create a Kerr black hole like Gargantua\n",
    "bh = bhsim.KerrBlackHole(mass=1.0, spin=0.99)\n",
    "\n",
    "# Print properties\n",
    "info = bh.info()\n",
    "print(\"╔════════════════════════════════════════════╗\")\n",
    "print(\"║        Kerr Black Hole Properties          ║\")\n",
    "print(\"╚════════════════════════════════════════════╝\")\n",
    "print(f\"\")\n",
    "print(f\"  Mass (M):              {info['mass']:.4f}\")\n",
    "print(f\"  Spin (a/M):            {info['spin']:.4f}\")\n",
    "print(f\"  Event Horizon (r+):    {info['horizon_radius']:.4f} M\")\n",
    "print(f\"  ISCO (prograde):       {info['isco_radius']:.4f} M\")\n",
    "print(f\"  ISCO (retrograde):     {info['isco_retrograde']:.4f} M\")\n",
    "print(f\"  Photon Sphere:         {info['photon_sphere']:.4f} M\")\n",
    "print(f\"  Shadow Radius:         {info['shadow_radius']:.4f} M\")\n",
    "print(f\"  Horizon Ω_H:           {info['omega_H']:.4f} c/M\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Comparing Schwarzschild vs Kerr\n",
    "\n",
    "Let's see how spin affects the key radii:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Compare different spins\n",
    "spins = np.linspace(0, 0.998, 50)\n",
    "horizons = []\n",
    "iscos = []\n",
    "photon_spheres = []\n",
    "\n",
    "for a in spins:\n",
    "    bh = bhsim.KerrBlackHole(mass=1.0, spin=a)\n",
    "    horizons.append(bh.horizon_radius)\n",
    "    iscos.append(bh.isco_radius)\n",
    "    photon_spheres.append(bh.photon_sphere_radius)\n",
    "\n",
    "# Plot\n",
    "fig, ax = plt.subplots(figsize=(12, 8))\n",
    "\n",
    "ax.plot(spins, horizons, 'b-', linewidth=2, label='Event Horizon (r+)')\n",
    "ax.plot(spins, iscos, 'r-', linewidth=2, label='ISCO (prograde)')\n",
    "ax.plot(spins, photon_spheres, 'g-', linewidth=2, label='Photon Sphere')\n",
    "\n",
    "# Mark Schwarzschild values\n",
    "ax.axhline(y=2, color='b', linestyle='--', alpha=0.3)\n",
    "ax.axhline(y=6, color='r', linestyle='--', alpha=0.3)\n",
    "ax.axhline(y=3, color='g', linestyle='--', alpha=0.3)\n",
    "\n",
    "ax.set_xlabel('Spin Parameter (a/M)', fontsize=14)\n",
    "ax.set_ylabel('Radius (M)', fontsize=14)\n",
    "ax.set_title('Kerr Black Hole: Key Radii vs Spin', fontsize=16)\n",
    "ax.legend(loc='upper right', fontsize=12)\n",
    "ax.set_xlim(0, 1)\n",
    "ax.set_ylim(0, 7)\n",
    "ax.grid(True, alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Rendering an Image\n",
    "\n",
    "Now let's render a black hole image with an accretion disk."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create black hole\n",
    "bh = bhsim.KerrBlackHole(mass=1.0, spin=0.99)\n",
    "\n",
    "# Render image\n",
    "print(\"Rendering... (this may take a minute)\")\n",
    "image = bh.render(\n",
    "    resolution=256,\n",
    "    inclination=85,  # Near edge-on view (like Interstellar)\n",
    "    observer_distance=100,\n",
    "    fov=15\n",
    ")\n",
    "\n",
    "# Display\n",
    "fig, ax = plt.subplots(figsize=(12, 12))\n",
    "im = ax.imshow(image, cmap='inferno', origin='lower')\n",
    "ax.set_title('Kerr Black Hole (a/M = 0.99, θ = 85°)', fontsize=16)\n",
    "ax.axis('off')\n",
    "plt.colorbar(im, ax=ax, label='Intensity', shrink=0.8)\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Exploring Parameter Space\n",
    "\n",
    "Use the interactive sliders to explore how different parameters affect the appearance:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def render_interactive(spin=0.99, inclination=85, resolution=128):\n",
    "    \"\"\"Interactive black hole rendering\"\"\"\n",
    "    bh = bhsim.KerrBlackHole(mass=1.0, spin=spin)\n",
    "    image = bh.render(\n",
    "        resolution=resolution,\n",
    "        inclination=inclination,\n",
    "        observer_distance=100,\n",
    "        fov=15\n",
    "    )\n",
    "    \n",
    "    fig, ax = plt.subplots(figsize=(10, 10))\n",
    "    ax.imshow(image, cmap='inferno', origin='lower')\n",
    "    ax.set_title(f'a/M = {spin:.3f}, θ = {inclination}°', fontsize=14)\n",
    "    ax.axis('off')\n",
    "    plt.show()\n",
    "\n",
    "# Interactive widget\n",
    "interact(\n",
    "    render_interactive,\n",
    "    spin=FloatSlider(min=0, max=0.998, step=0.01, value=0.99, description='Spin (a/M)'),\n",
    "    inclination=IntSlider(min=0, max=90, step=5, value=85, description='Inclination (°)'),\n",
    "    resolution=IntSlider(min=64, max=256, step=32, value=128, description='Resolution')\n",
    ")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Physics Analysis\n",
    "\n",
    "### Doppler Beaming\n",
    "\n",
    "The asymmetric brightness of the accretion disk is due to **relativistic Doppler beaming**. Material moving toward us appears brighter (blueshifted), while material moving away appears dimmer (redshifted)."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Analyze intensity profile\n",
    "bh = bhsim.KerrBlackHole(mass=1.0, spin=0.99)\n",
    "image = bh.render(resolution=256, inclination=85)\n",
    "\n",
    "# Horizontal slice through center\n",
    "center = image.shape[0] // 2\n",
    "horizontal_profile = image[center, :]\n",
    "\n",
    "fig, axes = plt.subplots(1, 2, figsize=(16, 6))\n",
    "\n",
    "# Image with slice indicator\n",
    "axes[0].imshow(image, cmap='inferno', origin='lower')\n",
    "axes[0].axhline(y=center, color='white', linestyle='--', alpha=0.5)\n",
    "axes[0].set_title('Black Hole Image', fontsize=14)\n",
    "axes[0].axis('off')\n",
    "\n",
    "# Intensity profile\n",
    "x = np.linspace(-10, 10, len(horizontal_profile))\n",
    "axes[1].plot(x, horizontal_profile, 'orange', linewidth=2)\n",
    "axes[1].fill_between(x, horizontal_profile, alpha=0.3, color='orange')\n",
    "axes[1].set_xlabel('Position (M)', fontsize=12)\n",
    "axes[1].set_ylabel('Intensity', fontsize=12)\n",
    "axes[1].set_title('Horizontal Intensity Profile (Doppler Asymmetry)', fontsize=14)\n",
    "axes[1].grid(True, alpha=0.3)\n",
    "\n",
    "# Mark the shadow\n",
    "axes[1].axvspan(-2.5, 2.5, alpha=0.2, color='black', label='Shadow region')\n",
    "axes[1].legend()\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "# Calculate asymmetry\n",
    "left_half = np.sum(image[:, :image.shape[1]//2])\n",
    "right_half = np.sum(image[:, image.shape[1]//2:])\n",
    "asymmetry = (left_half - right_half) / (left_half + right_half)\n",
    "print(f\"\\nIntensity asymmetry: {asymmetry:.2%}\")\n",
    "print(\"(Positive = left side brighter due to approaching material)\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Gravitational Lensing\n",
    "\n",
    "The black hole bends light, creating:\n",
    "- **Einstein ring**: A bright ring at the photon sphere radius\n",
    "- **Shadow**: The dark region where light cannot escape\n",
    "- **Secondary images**: Light that orbits the black hole before reaching us"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Compare face-on vs edge-on views\n",
    "fig, axes = plt.subplots(1, 3, figsize=(18, 6))\n",
    "\n",
    "for i, inc in enumerate([5, 45, 85]):\n",
    "    bh = bhsim.KerrBlackHole(mass=1.0, spin=0.99)\n",
    "    image = bh.render(resolution=256, inclination=inc)\n",
    "    \n",
    "    axes[i].imshow(image, cmap='inferno', origin='lower')\n",
    "    axes[i].set_title(f'Inclination = {inc}°', fontsize=14)\n",
    "    axes[i].axis('off')\n",
    "\n",
    "plt.suptitle('Effect of Viewing Angle on Black Hole Appearance', fontsize=16, y=1.02)\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Save High-Resolution Image"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Render and save a high-resolution image\n",
    "print(\"Rendering high-resolution image (512x512)...\")\n",
    "bh = bhsim.KerrBlackHole(mass=1.0, spin=0.998)\n",
    "image = bh.render(resolution=512, inclination=85)\n",
    "\n",
    "# Save\n",
    "bhsim.save_image(image, 'gargantua_hires.png', colormap='inferno')\n",
    "print(\"Saved to gargantua_hires.png\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "---\n",
    "\n",
    "## References\n",
    "\n",
    "1. Kerr, R.P. (1963). *Gravitational field of a spinning mass as an example of algebraically special metrics*\n",
    "2. James, O. et al. (2015). *Gravitational lensing by spinning black holes in astrophysics, and in the movie Interstellar*\n",
    "3. Event Horizon Telescope Collaboration (2019). *First M87 Event Horizon Telescope Results*\n",
    "\n",
    "---\n",
    "\n",
    "*Made with BlackHole-Sim* 🌌"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.10.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}